# Data Gathering libraries

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests
import numpy as np
from fake_useragent import UserAgent
# Create a UserAgent object
ua = UserAgent()

# Set the header and url

In [2]:
url="https://www.imdb.com/search/title/?genres=drama&countries=PK"
headers = {
    'User-Agent': ua.random,
    'Referer': 'https://www.example.com/',
    'Accept-Language': 'en-US,en;q=0.9',
}

# Function for getting the data

In [3]:
# Function to scrape data
def scrape_imdb_data(url):
    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome()

    try:
        # Open the IMDb URL
        driver.get(url)
#         response = driver.page_source
#         soup = BeautifulSoup(response, "html.parser")
#         titles = [title.text.strip() for title in soup.find_all('h3')]

        # Wait for the "Load More" button to appear
        print("Reaching the load more button")
        button_xpath = "//div[@class='sc-619d2eab-0 fOxpqs']//div[@class='sc-7d0fab22-0 fXtoQb pagination-container']//button[@class='ipc-btn ipc-btn--single-padding ipc-btn--center-align-content ipc-btn--default-height ipc-btn--core-base ipc-btn--theme-base ipc-btn--on-accent2 ipc-text-button ipc-see-more__button']"
        load_more_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))

        # Scroll into view
        ActionChains(driver).move_to_element(load_more_button).perform()
        
        time.sleep(2)
        print("Clicking the button")
#         load_more_button.click()
#         print("Button click done")
#         print("Loading more content")

        # Keep clicking the "Load More" button until it disappears
        try:
            print("Inside the try block")
            while load_more_button.is_displayed():
                print("Clicking")
                ActionChains(driver).move_to_element(load_more_button).perform()
                load_more_button.click()
                print("click done")
                time.sleep(10)  # Add a short delay to allow the content to load

        except Exception as e:
            print(e,"Not clickable")

        # Once all data is loaded, get the page source
        page_source = driver.page_source

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')
        
        box=soup.find_all("li",class_="ipc-metadata-list-summary-item")
        # Extract and print the data you need
#         titles = [title.text.strip() for title in soup.find_all('h3')]
        title = [i.find("h3").text if i.find("h3") else np.nan for i in box]
        
        urls = [i.find("a").get("href") if i.find("a") else np.nan for i in box]
        
        release_year = [i.find("span", class_="sc-be6f1408-8 fcCUPU dli-title-metadata-item").text if i.find("span", class_="sc-be6f1408-8 fcCUPU dli-title-metadata-item") else np.nan for i in box]
        rating = [i.find("span", class_="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating").text.split()[0] if i.find("span", class_="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating") else np.nan for i in box]
        votes = [i.find("span", class_="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating").text.split()[1] if i.find("span", class_="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating") else np.nan for i in box]
        title_type = [i.find("span", class_="sc-be6f1408-3 bpSATy dli-title-type-data").text if i.find("span", class_="sc-be6f1408-3 bpSATy dli-title-type-data") else np.nan for i in box]
        desc = [i.find("div", class_="ipc-html-content-inner-div").text if i.find("div", class_="ipc-html-content-inner-div") else np.nan for i in box]
        drama_poster = [i.find("img", class_="ipc-image").get("src") if i.find("img", class_="ipc-image") else np.nan for i in box]


        # Add more data extraction as needed

        return {"Title":title,
                "URL":urls,
                "release_year":release_year,
                "rating":rating,
                "votes":votes,
               "type":title_type,
                "des":desc,
                "pos":drama_poster
               }

    finally:
        # Close the browser window
        driver.quit()

In [4]:
#  Call the function to scrape IMDb data
result = scrape_imdb_data(url)

# Print the result
print(result)

Reaching the load more button
Clicking the button
Inside the try block
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Clicking
click done
Message: stale element reference: stale element not found
  (Session info: chrome=121.0.6167.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-elemen

In [5]:
print(len(result["URL"]))

1667


# Now getting the genur and Stars

In [6]:
crew_url=[]
for i in result["URL"]:
    crew_url.append(url[:20]+i)

In [9]:
each_crew = []
each_stars = []

# Assuming df is your DataFrame with the column "URL"
counter=0
for url in crew_url:
    print(counter)
    one_one_crew = []
    one_one_stars = []

    try:
        # Make an HTTP request
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an HTTPError for bad responses

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        if counter==8520:
            break
        else:
            # Extract crew information
            all_crew = soup.find_all("span", class_="ipc-chip__text")
            for crew_member in all_crew:
                try:
    #             print(crew_member.text)
                    one_one_crew.append(crew_member.text)
                except:
                    one_one_crew.append(np.nan)
            each_crew.append(one_one_crew)

            # Extract stars information
            stars = soup.find_all("div", class_="sc-bfec09a1-5 hNfYaW")
            for star in stars:
                try:
                    star_name = star.find("a", class_="sc-bfec09a1-1 gCQkeh")
                    if star_name:
        #                 print(star_name.text)
                        one_one_stars.append(star_name.text)
                except:
                    one_one_stars.append(np.nan)
            each_stars.append(one_one_stars)
            counter+=1

    except requests.exceptions.RequestException as e:
        print(f"Error during request: {e}")

0
1
2
Error during request: 504 Server Error: Gateway Time-out for url: https://www.imdb.com/title/tt21876438/?ref_=sr_i_3
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247

1590
1591
1592
1593
1594
1595
1596
1597
1598
1599
1600
1601
1602
1603
1604
1605
1606
1607
1608
1609
1610
1611
1612
1613
1614
1615
1616
1617
1618
1619
1620
1621
1622
1623
1624
1625
1626
1627
1628
Error during request: 504 Server Error: Gateway Time-out for url: https://www.imdb.com/title/tt5375500/?ref_=sr_i_1640
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654


In [11]:
len(each_crew)

1655

In [12]:
len(each_stars)

1655

In [14]:
result["crews"]=each_crew

In [16]:
result["stars"]=each_stars

# Now result should contain

In [18]:
result.keys()

dict_keys(['Title', 'URL', 'release_year', 'rating', 'votes', 'type', 'des', 'pos', 'crews', 'stars'])

In [38]:
result["Title"]=result["Title"][:1655]

In [39]:
result["URL"]=result["URL"][:1655]

In [40]:
result["des"]=result["des"][:1655]

In [41]:
result["release_year"]=result["release_year"][:1655]

In [42]:
result["votes"]=result["votes"][:1655]

In [43]:
result["pos"]=result["pos"][:1655] 

In [44]:
result["type"]=result["type"][:1655]

In [45]:
result["rating"]=result["rating"][:1655]

In [46]:
print(len(result["crews"]))
print(len(result["Title"]))

1655
1655


# Convert into Dataframe

In [48]:
df=pd.DataFrame(result)

In [50]:
df.to_csv("Pakistani_dramas.csv",index=False)